# Deep MNIST for Experts
[original](https://www.tensorflow.org/get_started/mnist/pros)

## Data

The MNIST data is split into three parts: 55,000 data points of training data (mnist.train), 10,000 points of test data (mnist.test), and 5,000 points of validation data (mnist.validation)

Both the training set and test set contain images and their corresponding labels; for example the training images are mnist.train.images and the training labels are mnist.train.labels.

Each image is 28 pixels by 28 pixels. We can flatten this array into a vector of 28x28 = 784 numbers.

In [1]:
import argparse
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from typing import Optional, Iterable, Any
from types import SimpleNamespace


def init_data():
    global mnist
    mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)


def init_model():
    global x, y, dropout_keep_prob
    
    def weight_variable(shape):
        """weight_variable generates a weight variable of a given shape."""
        return tf.get_variable('weight', shape, initializer=tf.truncated_normal_initializer(stddev=0.1))
    
    def bias_variable(shape):
        """bias_variable generates a bias variable of a given shape."""
        return tf.get_variable('bias', shape, initializer=tf.constant_initializer(0.1))
    
    def conv_relu(inputs, kernel_shape, bias_shape):
        W = weight_variable(kernel_shape)
        b = bias_variable(bias_shape)
        conv = tf.nn.conv2d(inputs, W, strides=[1, 1, 1, 1], padding='SAME')
        return tf.nn.relu(conv + b)
    
    def max_pool_2x2(inputs):
        """max_pool_2x2 downsamples a feature map by 2X."""
        return tf.nn.max_pool(inputs, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    def fc_relu(inputs, weight_shape, bias_shape):
        W = weight_variable(weight_shape)
        b = bias_variable(bias_shape)
        return tf.nn.relu(tf.matmul(inputs, W) + b)
    
    def linear(inputs, weight_shape, bias_shape):
        W = weight_variable(weight_shape)
        b = bias_variable(bias_shape)
        return tf.matmul(fc1_drop, W) + b
    
    x = tf.placeholder(tf.float32, [None, 784])
    
    # Reshape to use within a convolutional neural net.
    # Last dimension is for "features" - there is only one here, since images are
    # grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
    global x_image
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    
    with tf.variable_scope("conv_maxpool_1"):
        relu1 = conv_relu(x_image, [5, 5, 1, 32], [32])
        max_pool1 = max_pool_2x2(relu1)
    with tf.variable_scope("conv_maxpool_2"):
        relu2 = conv_relu(max_pool1, [5, 5, 32, 64], [64])
        global max_pool2
        max_pool2 = max_pool_2x2(relu2)
    with tf.variable_scope("fully_connected"):
        # after 2 round of downsampling, our 28x28 image
        # is down to 7x7x64 feature maps -- map this to 1024 features
        global max_pool2_flat
        max_pool2_flat = tf.reshape(max_pool2, [-1, 7 * 7 * 64])
        fc1 = fc_relu(max_pool2_flat, [7 * 7 * 64, 1024], [1024])
    with tf.name_scope("dropout"):
        # Dropout - controls the complexity of the model, prevents co-adaptation of features
        dropout_keep_prob = tf.placeholder(tf.float32)
        fc1_drop = tf.nn.dropout(fc1, dropout_keep_prob)
    with tf.variable_scope("output"):
        # Map the 1024 features to 10 classes, one for each digit
        y = linear(fc1_drop, [1024, 10], [10])


def init_train():
    global y_, loss, train_op
    y_ = tf.placeholder(tf.float32, [None, 10])
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
    train_op = tf.train.AdamOptimizer(1e-4).minimize(loss)


def train():
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch_xs, batch_ys = mnist.train.next_batch(50)
        if i % 100 == 0:
            log_model_status(batch_xs, batch_ys, i)
        sess.run(train_op, feed_dict={x: batch_xs,
                                      y_: batch_ys,
                                      dropout_keep_prob: 0.5})


def init_evaluate():
    global correct_prediction, accuracy
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


def log_model_status(inputs, labels, step):
    acc = sess.run(accuracy, feed_dict={x: inputs,
                                        y_: labels,
                                        dropout_keep_prob: 1.0})
    print('step %d, training accuracy %g' % (step, acc))


def evaluate():
    acc = sess.run(accuracy, feed_dict={x: mnist.test.images,
                                        y_: mnist.test.labels,
                                        dropout_keep_prob: 1.0})
    print("test accuracy is: ".format(acc))
    
    
def init_flags(flags: Optional[dict] = None):
    global FLAGS
    
    if flags is None:
        parser = argparse.ArgumentParser()
        parser.add_argument("--prepare", dest='just_data', action="store_true")
        parser.add_argument("--evaluate_only", action="store_true")
        parser.add_argument('--data_dir',
                            type=str,
                            default='/tmp/tensorflow/mnist/input_data',
                            help='Directory for storing input data')
        FLAGS, _ = parser.parse_known_args()
    else:
        FLAGS = SimpleNamespace(**flags)


def main(flags: Optional[dict] = None):
    init_flags(flags)
    init_data()

    if FLAGS.just_data:
        exit()
    
    global sess
    sess = tf.Session()
    
    init_model()
    init_train()
    init_evaluate()
    
    train()
    evaluate()


In [3]:
batch_xs, batch_ys = mnist.train.next_batch(50)

In [14]:
a = sess.run(max_pool2, feed_dict={x: batch_xs,
                           y_: batch_ys,
                           dropout_keep_prob: 1.0})

In [15]:
a.shape

(50, 28, 28, 64)

In [ ]:
main()

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.08
step 100, training accuracy 0.84
step 200, training accuracy 0.9
step 300, training accuracy 0.86
step 400, training accuracy 0.92
step 500, training accuracy 0.94
step 600, training accuracy 0.98
step 700, training accuracy 0.94
step 800, training accuracy 0.98
step 900, training accuracy 0.92
step 1000, training accuracy 0.96
step 1100, training accuracy 0.98
step 1200, training accuracy 0.96
step 1300, training accuracy 0.96
step 1400, training accuracy 0.98
step 1500, training accuracy 0.98
step 1600, training accuracy 0.98
step 1700, training accuracy 0.94
step 1800, training accuracy 0.96
step 1900, training accuracy 0.98
step 2000, training accuracy 0.96
step 2100, traini